# Task-2: WSI Patch Generator
###M. Jawad Bashir (MSDS-20) - 330295

In [ ]:
!apt update && apt install -y openslide-tools
!pip install openslide-python

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Downsampling Factor getter

In [ ]:
def get_downsampling_factor(res_level):
  if res_level == 0:
    return 1
  elif res_level == 1:
    return 4
  elif res_level == 2:
    return 16
  elif res_level == 3:
    return 32

###Threshold check for white WSI regions

In [ ]:
def get_patch_ratio(ptch):
  ptch = ptch.convert('L')
  ptch = np.asarray(ptch)
  h, _ = np.histogram(ptch, bins=255, range=(0, 255))
  ratio = h[np.argmax(h)-1:np.argmax(h)+1].sum()/h.sum()
  if ratio > 0.5:
    print("White Patch")
    return False
  else:
    print("True Patch")
    return True

In [ ]:
import os
from os.path import isfile, join
import sys
import openslide
import numpy as np
from math import ceil
from PIL import Image

###Patch Extractor Function

In [ ]:
def patch_extractor(svs_path, out_path,res_level,patch_size):
    svs = openslide.OpenSlide(svs_path)
    print('Slide Properties:', svs.properties)
    print('Slide Levels:', svs.level_count)
    width,height = svs.level_dimensions[res_level]
    print('Slide Dimension: ', width, height)
    #print(svs.get_best_level_for_downsample(4))
      
    x_poses = np.arange(0,width, patch_size)
    y_poses = np.arange(0,height, patch_size)

    total_patches = len(x_poses)*len(y_poses)
    patch_no = 1

    for i, xstart_pos in enumerate(x_poses):
      for j, ystart_pos in enumerate(y_poses):
            
            xend_pos = min(width, xstart_pos + patch_size)
            yend_pos = min(height, ystart_pos + patch_size)

            #print("Endposes: ",xend_pos,yend_pos)

            patch_width = xend_pos - xstart_pos
            patch_height = yend_pos - ystart_pos
            downsampling_factor = get_downsampling_factor(res_level)
            patch = svs.read_region((xstart_pos * downsampling_factor, ystart_pos * downsampling_factor),
                                    res_level,
                                    (patch_width, patch_height))
            #threshold_check
            if get_patch_ratio(patch):
                patch.load()
                patch_rgb = Image.new("RGB", patch.size, (255, 255, 255))
                patch_rgb.paste(patch, mask=patch.split()[3])
                print("\n")
                print("Patch Data: ", xstart_pos, ystart_pos, "Patch Size:", patch.size)
                print("Patch No: " + str(patch_no))

                outp_name = join(out_path,'patch_' + str(i) + str(j) + '.jpg')
                
                patch_rgb.save(outp_name)

                print("Tile", patch_no, "/", total_patches, "created")
                patch_no = patch_no + 1

### Patch Generation 1:
#### Settings:
Patch Size = 4096x4096, Magnification: 40x (level 0)

In [ ]:
wsi_path = "/content/drive/MyDrive/WSI_Data/TCGA-D8-A1JM-01Z-00-DX1.3CD15BC8-08B3-4BEE-9FD7-A8BEED33268E.svs"
output_folder = "/content/drive/MyDrive/WSI_Data/Patches_4096_40x"
patch_extractor(wsi_path, output_folder, 0, 4096)

Slide Properties: <_PropertyMap {'aperio.AppMag': '40', 'aperio.DSR ID': 'resc3-dsr2', 'aperio.Date': '05/18/11', 'aperio.DisplayColor': '0', 'aperio.Exposure Scale': '0.000001', 'aperio.Exposure Time': '109', 'aperio.Filename': 'TCGA-D8-A1JM-01Z-00-DX1', 'aperio.Focus Offset': '0.000000', 'aperio.ICC Profile': 'ScanScope v1', 'aperio.ImageID': '65791', 'aperio.Left': '21.836823', 'aperio.LineAreaXOffset': '0.003267', 'aperio.LineAreaYOffset': '-0.003095', 'aperio.LineCameraSkew': '0.001071', 'aperio.MPP': '0.2480', 'aperio.OriginalHeight': '79299', 'aperio.OriginalWidth': '80352', 'aperio.Parmset': 'GOG136 on RESBPCLACIE01', 'aperio.ScanScope ID': 'SS1511CNTLR', 'aperio.StripeWidth': '992', 'aperio.Time': '17:46:20', 'aperio.Time Zone': 'GMT-04:00', 'aperio.Title': 'TCGA-D8-A1JM-01Z-00-DX1', 'aperio.Top': '23.566193', 'aperio.User': 'ef0ab3c3-0a83-453b-9734-aa4086e297ef', 'openslide.comment': 'Aperio Image Library v10.2.41\r\n80352x79299 [0,100 77111x79199] (240x240) JPEG/RGB Q=30|App

### Patch Generation 2:
#### Settings:
Patch Size = 1024x1024, Magnification: 20x (level 1)

In [ ]:
wsi_path = "/content/drive/MyDrive/WSI_Data/TCGA-D8-A1JM-01Z-00-DX1.3CD15BC8-08B3-4BEE-9FD7-A8BEED33268E.svs"
output_folder = "/content/drive/MyDrive/WSI_Data/Patches_1024_20x"
patch_extractor(wsi_path, output_folder, 1, 1024)

Slide Properties: <_PropertyMap {'aperio.AppMag': '40', 'aperio.DSR ID': 'resc3-dsr2', 'aperio.Date': '05/18/11', 'aperio.DisplayColor': '0', 'aperio.Exposure Scale': '0.000001', 'aperio.Exposure Time': '109', 'aperio.Filename': 'TCGA-D8-A1JM-01Z-00-DX1', 'aperio.Focus Offset': '0.000000', 'aperio.ICC Profile': 'ScanScope v1', 'aperio.ImageID': '65791', 'aperio.Left': '21.836823', 'aperio.LineAreaXOffset': '0.003267', 'aperio.LineAreaYOffset': '-0.003095', 'aperio.LineCameraSkew': '0.001071', 'aperio.MPP': '0.2480', 'aperio.OriginalHeight': '79299', 'aperio.OriginalWidth': '80352', 'aperio.Parmset': 'GOG136 on RESBPCLACIE01', 'aperio.ScanScope ID': 'SS1511CNTLR', 'aperio.StripeWidth': '992', 'aperio.Time': '17:46:20', 'aperio.Time Zone': 'GMT-04:00', 'aperio.Title': 'TCGA-D8-A1JM-01Z-00-DX1', 'aperio.Top': '23.566193', 'aperio.User': 'ef0ab3c3-0a83-453b-9734-aa4086e297ef', 'openslide.comment': 'Aperio Image Library v10.2.41\r\n80352x79299 [0,100 77111x79199] (240x240) JPEG/RGB Q=30|App

## Observations:



1.   Setting-1 (4096x4096 @ 40x) patches are in higher resolution and cellular level features are very distanct and separatable. These settings are best for cellular level feature extraction i.e. mitosis detections and counting.
2.   Setting-2 (1024x1024 @ 20x) patches are in lower resolution as they are downsampled by a factor of 4 (level-1). This setting seems to be good for identifying obvious tissue patterns and segmentation tasks i.e. lymphocites segmetation
3.  Thus concluded, task-based magnification and downsampling settings should be well verified before patch extraction

